## Imports

## Créer un modèle prédictif

In [ ]:
# tout les k-means vont ici

## Créer un modèle de recommandation aléatoire

## Comparer le score du modèle prédictif à celui du modèle aléatoire

In [ ]:
# dans une fonction qui s'occupe des tests